In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
class Arg:
    arch = 'resnet18'
    data = '/data/imagenet/ILSVRC/Data/CLS-LOC/'
    batch_size = 128
    workers = 4
    start_epoch = 0
    epochs = 90
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    print_freq = 10
    resume = ''
    world_size = -1
    rank = -1
    dist_url = 'tcp://224.66.41.62:23456'
    seed = None
    dist_backend = 'nccl'
    gpu = None
    distributed = False
    pretrained = False
    evaluate = False
    multiprocessing_distributed = False
args = Arg()

In [3]:
best_acc1 = 0

In [4]:
ngpus_per_node = torch.cuda.device_count()

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if args.gpu is not None:
                images = images.cuda(args.gpu, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [7]:
def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [10]:
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()

    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
    elif args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.resume, map_location=loc)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = '/home/jtang/robustness/imagenet_r/DeepAugment/EDSR_real_train'
    valdir = os.path.join(args.data, 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                and args.rank % ngpus_per_node == 0):
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)

In [11]:
main_worker(args.gpu, ngpus_per_node, args)

=> creating model 'resnet18'
Epoch: [0][   0/1002]	Time  3.918 ( 3.918)	Data  1.728 ( 1.728)	Loss 6.9459e+00 (6.9459e+00)	Acc@1   0.00 (  0.00)	Acc@5   1.56 (  1.56)
Epoch: [0][  10/1002]	Time  0.339 ( 0.646)	Data  0.000 ( 0.159)	Loss 7.5724e+00 (7.1461e+00)	Acc@1   0.00 (  0.21)	Acc@5   0.00 (  0.64)
Epoch: [0][  20/1002]	Time  0.334 ( 0.500)	Data  0.000 ( 0.094)	Loss 7.2786e+00 (7.2534e+00)	Acc@1   0.00 (  0.15)	Acc@5   0.78 (  0.56)
Epoch: [0][  30/1002]	Time  0.338 ( 0.451)	Data  0.163 ( 0.102)	Loss 7.1802e+00 (7.2313e+00)	Acc@1   1.56 (  0.20)	Acc@5   2.34 (  0.71)
Epoch: [0][  40/1002]	Time  0.340 ( 0.443)	Data  0.000 ( 0.118)	Loss 6.9393e+00 (7.1908e+00)	Acc@1   0.00 (  0.21)	Acc@5   1.56 (  0.74)
Epoch: [0][  50/1002]	Time  0.337 ( 0.430)	Data  0.027 ( 0.117)	Loss 7.0323e+00 (7.1531e+00)	Acc@1   0.78 (  0.21)	Acc@5   1.56 (  0.74)
Epoch: [0][  60/1002]	Time  0.344 ( 0.424)	Data  0.000 ( 0.120)	Loss 6.9549e+00 (7.1194e+00)	Acc@1   0.00 (  0.18)	Acc@5   1.56 (  0.69)
Epoch: [0][ 

Epoch: [0][ 600/1002]	Time  0.392 ( 0.389)	Data  0.000 ( 0.089)	Loss 6.7248e+00 (6.8309e+00)	Acc@1   0.00 (  0.34)	Acc@5   3.91 (  1.51)
Epoch: [0][ 610/1002]	Time  0.389 ( 0.389)	Data  0.000 ( 0.088)	Loss 6.7343e+00 (6.8290e+00)	Acc@1   0.00 (  0.35)	Acc@5   0.00 (  1.53)
Epoch: [0][ 620/1002]	Time  0.383 ( 0.389)	Data  0.000 ( 0.088)	Loss 6.7356e+00 (6.8268e+00)	Acc@1   0.00 (  0.35)	Acc@5   2.34 (  1.54)
Epoch: [0][ 630/1002]	Time  0.395 ( 0.389)	Data  0.000 ( 0.088)	Loss 6.6156e+00 (6.8242e+00)	Acc@1   0.78 (  0.35)	Acc@5   4.69 (  1.56)
Epoch: [0][ 640/1002]	Time  0.388 ( 0.389)	Data  0.000 ( 0.088)	Loss 6.6035e+00 (6.8219e+00)	Acc@1   0.78 (  0.36)	Acc@5   2.34 (  1.57)
Epoch: [0][ 650/1002]	Time  0.380 ( 0.389)	Data  0.000 ( 0.087)	Loss 6.6506e+00 (6.8197e+00)	Acc@1   0.00 (  0.36)	Acc@5   3.12 (  1.59)
Epoch: [0][ 660/1002]	Time  0.381 ( 0.389)	Data  0.000 ( 0.087)	Loss 6.6265e+00 (6.8173e+00)	Acc@1   2.34 (  0.36)	Acc@5   3.12 (  1.61)
Epoch: [0][ 670/1002]	Time  0.384 ( 0.389

Test: [240/391]	Time  1.162 ( 0.385)	Loss 6.1151e+00 (6.4388e+00)	Acc@1   0.00 (  0.98)	Acc@5  12.50 (  3.92)
Test: [250/391]	Time  0.115 ( 0.382)	Loss 6.2856e+00 (6.4358e+00)	Acc@1   1.56 (  1.03)	Acc@5   7.81 (  4.06)
Test: [260/391]	Time  0.965 ( 0.384)	Loss 6.4512e+00 (6.4399e+00)	Acc@1   0.78 (  1.00)	Acc@5   5.47 (  4.00)
Test: [270/391]	Time  0.119 ( 0.383)	Loss 6.6322e+00 (6.4421e+00)	Acc@1   0.00 (  1.02)	Acc@5   1.56 (  3.98)
Test: [280/391]	Time  0.839 ( 0.382)	Loss 6.7209e+00 (6.4470e+00)	Acc@1   0.00 (  0.98)	Acc@5   0.78 (  3.90)
Test: [290/391]	Time  0.116 ( 0.380)	Loss 6.4970e+00 (6.4489e+00)	Acc@1   0.00 (  0.96)	Acc@5   1.56 (  3.84)
Test: [300/391]	Time  0.945 ( 0.381)	Loss 6.5854e+00 (6.4518e+00)	Acc@1   0.00 (  0.94)	Acc@5   0.00 (  3.79)
Test: [310/391]	Time  0.114 ( 0.380)	Loss 6.3602e+00 (6.4555e+00)	Acc@1   0.00 (  0.92)	Acc@5   5.47 (  3.73)
Test: [320/391]	Time  0.978 ( 0.382)	Loss 6.4369e+00 (6.4548e+00)	Acc@1   0.00 (  0.89)	Acc@5   1.56 (  3.69)
Test: [330

Epoch: [1][ 470/1002]	Time  0.392 ( 0.387)	Data  0.000 ( 0.031)	Loss 6.3451e+00 (6.3833e+00)	Acc@1   0.78 (  1.21)	Acc@5   6.25 (  4.63)
Epoch: [1][ 480/1002]	Time  0.377 ( 0.387)	Data  0.000 ( 0.031)	Loss 6.3178e+00 (6.3820e+00)	Acc@1   0.78 (  1.21)	Acc@5   3.12 (  4.63)
Epoch: [1][ 490/1002]	Time  0.379 ( 0.387)	Data  0.000 ( 0.031)	Loss 6.2471e+00 (6.3792e+00)	Acc@1   1.56 (  1.22)	Acc@5   7.03 (  4.66)
Epoch: [1][ 500/1002]	Time  0.384 ( 0.387)	Data  0.000 ( 0.030)	Loss 6.2271e+00 (6.3774e+00)	Acc@1   0.78 (  1.22)	Acc@5   7.81 (  4.68)
Epoch: [1][ 510/1002]	Time  0.375 ( 0.387)	Data  0.000 ( 0.030)	Loss 6.3344e+00 (6.3750e+00)	Acc@1   0.78 (  1.24)	Acc@5   4.69 (  4.71)
Epoch: [1][ 520/1002]	Time  0.382 ( 0.387)	Data  0.000 ( 0.030)	Loss 6.2055e+00 (6.3731e+00)	Acc@1   3.12 (  1.24)	Acc@5  10.94 (  4.74)
Epoch: [1][ 530/1002]	Time  0.379 ( 0.387)	Data  0.000 ( 0.030)	Loss 6.2582e+00 (6.3700e+00)	Acc@1   0.78 (  1.26)	Acc@5   4.69 (  4.77)
Epoch: [1][ 540/1002]	Time  0.386 ( 0.387

Test: [ 80/391]	Time  0.238 ( 0.375)	Loss 6.1282e+00 (5.8709e+00)	Acc@1   1.56 (  2.60)	Acc@5   5.47 (  9.88)
Test: [ 90/391]	Time  0.755 ( 0.381)	Loss 6.6221e+00 (5.9188e+00)	Acc@1   0.00 (  2.40)	Acc@5   0.00 (  9.21)
Test: [100/391]	Time  0.116 ( 0.372)	Loss 5.5116e+00 (5.9552e+00)	Acc@1   3.12 (  2.21)	Acc@5  16.41 (  8.62)
Test: [110/391]	Time  1.202 ( 0.381)	Loss 6.1676e+00 (5.9617e+00)	Acc@1   0.78 (  2.23)	Acc@5   1.56 (  8.39)
Test: [120/391]	Time  0.110 ( 0.378)	Loss 5.8614e+00 (5.9313e+00)	Acc@1   0.00 (  2.36)	Acc@5   0.78 (  8.64)
Test: [130/391]	Time  1.064 ( 0.384)	Loss 5.7235e+00 (5.9181e+00)	Acc@1   4.69 (  2.46)	Acc@5  12.50 (  8.75)
Test: [140/391]	Time  0.246 ( 0.377)	Loss 6.1694e+00 (5.9222e+00)	Acc@1   0.00 (  2.42)	Acc@5   1.56 (  8.48)
Test: [150/391]	Time  0.909 ( 0.382)	Loss 6.0138e+00 (5.9174e+00)	Acc@1   3.12 (  2.44)	Acc@5  10.94 (  8.47)
Test: [160/391]	Time  0.352 ( 0.378)	Loss 5.7797e+00 (5.9063e+00)	Acc@1   0.00 (  2.45)	Acc@5   9.38 (  8.68)
Test: [170

...

Test: [ 30/391]	Time  0.116 ( 0.363)	Loss 3.4783e+00 (2.6209e+00)	Acc@1  39.06 ( 47.30)	Acc@5  64.06 ( 71.62)
Test: [ 40/391]	Time  0.349 ( 0.345)	Loss 2.7173e+00 (2.4043e+00)	Acc@1  41.41 ( 50.82)	Acc@5  67.19 ( 74.45)
Test: [ 50/391]	Time  0.118 ( 0.344)	Loss 1.7225e+00 (2.4923e+00)	Acc@1  60.16 ( 49.63)	Acc@5  83.59 ( 73.13)
Test: [ 60/391]	Time  0.118 ( 0.340)	Loss 2.2563e+00 (2.3825e+00)	Acc@1  42.19 ( 51.37)	Acc@5  76.56 ( 74.45)
Test: [ 70/391]	Time  0.696 ( 0.334)	Loss 2.2699e+00 (2.4159e+00)	Acc@1  41.41 ( 49.85)	Acc@5  70.31 ( 73.99)
Test: [ 80/391]	Time  0.117 ( 0.328)	Loss 2.6458e+00 (2.4512e+00)	Acc@1  39.06 ( 48.37)	Acc@5  69.53 ( 73.56)
Test: [ 90/391]	Time  0.631 ( 0.331)	Loss 2.2308e+00 (2.4420e+00)	Acc@1  43.75 ( 47.91)	Acc@5  81.25 ( 73.78)
Test: [100/391]	Time  0.119 ( 0.323)	Loss 2.2701e+00 (2.4157e+00)	Acc@1  41.41 ( 47.86)	Acc@5  78.12 ( 74.26)
Test: [110/391]	Time  0.484 ( 0.324)	Loss 2.4842e+00 (2.4159e+00)	Acc@1  39.84 ( 47.64)	Acc@5  75.78 ( 74.31)
Test: [120

Epoch: [88][ 300/1002]	Time  0.424 ( 0.391)	Data  0.000 ( 0.033)	Loss 1.6329e+00 (1.5143e+00)	Acc@1  60.94 ( 67.44)	Acc@5  82.81 ( 84.01)
Epoch: [88][ 310/1002]	Time  0.421 ( 0.392)	Data  0.019 ( 0.033)	Loss 1.7091e+00 (1.5112e+00)	Acc@1  64.84 ( 67.49)	Acc@5  80.47 ( 84.06)
Epoch: [88][ 320/1002]	Time  0.391 ( 0.393)	Data  0.000 ( 0.033)	Loss 1.3979e+00 (1.5106e+00)	Acc@1  70.31 ( 67.47)	Acc@5  85.94 ( 84.09)
Epoch: [88][ 330/1002]	Time  0.388 ( 0.392)	Data  0.000 ( 0.033)	Loss 1.4897e+00 (1.5104e+00)	Acc@1  66.41 ( 67.48)	Acc@5  85.16 ( 84.10)
Epoch: [88][ 340/1002]	Time  0.378 ( 0.393)	Data  0.000 ( 0.032)	Loss 1.5191e+00 (1.5118e+00)	Acc@1  70.31 ( 67.46)	Acc@5  82.03 ( 84.09)
Epoch: [88][ 350/1002]	Time  0.399 ( 0.393)	Data  0.000 ( 0.032)	Loss 1.4145e+00 (1.5113e+00)	Acc@1  69.53 ( 67.48)	Acc@5  85.94 ( 84.06)
Epoch: [88][ 360/1002]	Time  0.376 ( 0.393)	Data  0.000 ( 0.032)	Loss 1.6936e+00 (1.5087e+00)	Acc@1  60.94 ( 67.56)	Acc@5  78.91 ( 84.10)
Epoch: [88][ 370/1002]	Time  0.356

Epoch: [88][ 900/1002]	Time  0.382 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.5302e+00 (1.5171e+00)	Acc@1  64.06 ( 67.32)	Acc@5  85.94 ( 83.91)
Epoch: [88][ 910/1002]	Time  0.384 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.4450e+00 (1.5191e+00)	Acc@1  65.62 ( 67.29)	Acc@5  85.94 ( 83.88)
Epoch: [88][ 920/1002]	Time  0.386 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.4878e+00 (1.5189e+00)	Acc@1  68.75 ( 67.30)	Acc@5  85.16 ( 83.89)
Epoch: [88][ 930/1002]	Time  0.386 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.5062e+00 (1.5195e+00)	Acc@1  66.41 ( 67.29)	Acc@5  85.16 ( 83.88)
Epoch: [88][ 940/1002]	Time  0.381 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.3081e+00 (1.5198e+00)	Acc@1  66.41 ( 67.28)	Acc@5  89.06 ( 83.87)
Epoch: [88][ 950/1002]	Time  0.384 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.4094e+00 (1.5188e+00)	Acc@1  66.41 ( 67.30)	Acc@5  84.38 ( 83.88)
Epoch: [88][ 960/1002]	Time  0.394 ( 0.392)	Data  0.000 ( 0.041)	Loss 1.1072e+00 (1.5184e+00)	Acc@1  73.44 ( 67.31)	Acc@5  90.62 ( 83.89)
Epoch: [88][ 970/1002]	Time  0.384

Epoch: [89][ 170/1002]	Time  0.380 ( 0.396)	Data  0.000 ( 0.037)	Loss 1.4676e+00 (1.5109e+00)	Acc@1  67.97 ( 67.88)	Acc@5  85.16 ( 84.19)
Epoch: [89][ 180/1002]	Time  0.385 ( 0.395)	Data  0.000 ( 0.036)	Loss 1.1849e+00 (1.5073e+00)	Acc@1  70.31 ( 67.90)	Acc@5  90.62 ( 84.28)
Epoch: [89][ 190/1002]	Time  0.388 ( 0.395)	Data  0.000 ( 0.036)	Loss 1.6743e+00 (1.5072e+00)	Acc@1  61.72 ( 67.85)	Acc@5  82.03 ( 84.30)
Epoch: [89][ 200/1002]	Time  0.380 ( 0.394)	Data  0.000 ( 0.035)	Loss 1.7257e+00 (1.5105e+00)	Acc@1  63.28 ( 67.78)	Acc@5  77.34 ( 84.22)
Epoch: [89][ 210/1002]	Time  0.392 ( 0.394)	Data  0.000 ( 0.035)	Loss 1.4412e+00 (1.5114e+00)	Acc@1  67.19 ( 67.78)	Acc@5  83.59 ( 84.13)
Epoch: [89][ 220/1002]	Time  0.384 ( 0.393)	Data  0.000 ( 0.035)	Loss 1.4161e+00 (1.5118e+00)	Acc@1  66.41 ( 67.73)	Acc@5  85.16 ( 84.11)
Epoch: [89][ 230/1002]	Time  0.380 ( 0.393)	Data  0.000 ( 0.034)	Loss 1.2776e+00 (1.5103e+00)	Acc@1  77.34 ( 67.77)	Acc@5  89.84 ( 84.14)
Epoch: [89][ 240/1002]	Time  0.380

Epoch: [89][ 770/1002]	Time  0.381 ( 0.388)	Data  0.000 ( 0.029)	Loss 1.5803e+00 (1.5065e+00)	Acc@1  66.41 ( 67.73)	Acc@5  82.03 ( 83.98)
Epoch: [89][ 780/1002]	Time  0.383 ( 0.388)	Data  0.000 ( 0.029)	Loss 1.4796e+00 (1.5059e+00)	Acc@1  63.28 ( 67.73)	Acc@5  85.16 ( 83.98)
Epoch: [89][ 790/1002]	Time  0.375 ( 0.388)	Data  0.000 ( 0.029)	Loss 1.3374e+00 (1.5063e+00)	Acc@1  69.53 ( 67.73)	Acc@5  85.16 ( 83.97)
Epoch: [89][ 800/1002]	Time  0.374 ( 0.388)	Data  0.002 ( 0.029)	Loss 1.3806e+00 (1.5067e+00)	Acc@1  69.53 ( 67.71)	Acc@5  83.59 ( 83.97)
Epoch: [89][ 810/1002]	Time  0.386 ( 0.388)	Data  0.000 ( 0.029)	Loss 1.7544e+00 (1.5069e+00)	Acc@1  63.28 ( 67.71)	Acc@5  79.69 ( 83.96)
Epoch: [89][ 820/1002]	Time  0.394 ( 0.388)	Data  0.000 ( 0.029)	Loss 1.6463e+00 (1.5068e+00)	Acc@1  61.72 ( 67.71)	Acc@5  82.81 ( 83.96)
Epoch: [89][ 830/1002]	Time  0.395 ( 0.388)	Data  0.002 ( 0.029)	Loss 1.5261e+00 (1.5070e+00)	Acc@1  62.50 ( 67.70)	Acc@5  84.38 ( 83.96)
Epoch: [89][ 840/1002]	Time  0.380

In [5]:
import torchvision.models as torch_models
import torch
from collections import OrderedDict

model = torch_models.resnet18()
    
checkpoint = torch.load('/home/jtang/Desktop/original_model/checkpoint.pth.tar')